# 从kafka中不断取数据

In [1]:
import json, time
from confluent_kafka import Consumer, KafkaError

In [7]:
Consumer?

Init signature: Consumer(self, /, *args, **kwargs)
Docstring:     
A high-level Apache Kafka consumer

.. py:function:: Consumer(config)

Create a new Consumer instance using the provided configuration *dict* (including properties and callback functions). See :ref:`pythonclient_configuration` for more information.

:param dict config: Configuration properties. At a minimum, ``group.id`` **must** be set and ``bootstrap.servers`` **should** be set.
Type:           type
Subclasses:     DeserializingConsumer


In [2]:
information = {}
kafka = {}
kafka['http'] = '192.168.50.110:19092'
# kafka['topicCtrl'] = 'dc_abnormal_control'
kafka['event_topic'] = 'dc_event'
kafka['offset'] = 'earliest'
kafka['groupid'] = 'AnomalyDetectTest01'
information['kafka'] = kafka

In [3]:
information

{'kafka': {'http': '192.168.50.110:19092',
  'topicData': 'dc_event',
  'offset': 'earliest',
  'groupid': 'AnomalyDetectTest01'}}

In [4]:
with open('../infoamtion.json', 'w') as f:
    json.dump(information, f, indent = 2, sort_keys = True, ensure_ascii = False)

In [5]:
c = Consumer({
    'bootstrap.servers':kafka['http'], # kafka所在ip地址
    'group.id':time.time(),
    'enable.auto.commit':True, # 是否自动提交offset，设为True时，每隔一段时间就会提交一次offset
    'default.topic.config':{
        'auto.offset.reset':kafka['offset']
    }  
})
# c.subscribe([kafka['event_topic']]) # 为consumer分配分区

In [6]:
c.subscribe([kafka['event_topic']]) # 为consumer分配分区

In [ ]:
# 没有break会不停的取数据
while True:
    msg = c.poll(100)
    if msg is None:
        continue
    else:
        topicMsg = msg.topic()
        message = json.loads(msg.value())

___________

# 从kafka中取一段时间内的数据

In [12]:
from confluent_kafka import Consumer, KafkaError, TopicPartition, Message
import datetime, time, json

In [13]:
kafka = {}
kafka['http'] = '192.168.50.107:19092'
kafka['topicCtrl'] = 'dc_abnormal_control'
kafka['topicData'] = 'dc_abnormal_data'
kafka['offset'] = 'earliest'
kafka['groupid'] = 'AnomalyDetectTest01'

In [14]:
c = Consumer({
    'bootstrap.servers':kafka['http'], # kafka所在ip地址
    'group.id':time.time(),
    'enable.auto.commit':True, # 是否自动提交offset，设为True时，每隔一段时间就会提交一次offset
    'default.topic.config':{
        'auto.offset.reset':kafka['offset']
    }  
})

In [15]:
# 拉取昨天一天的数据，start_time、end_time这两个时间可以随便设置
now = datetime.datetime.now() - datetime.timedelta(days=1)
start_time = datetime.datetime.strptime(now.strftime('%Y-%m-%d 00:00:00'),'%Y-%m-%d %H:%M:%S')
end_time = datetime.datetime.strptime(now.strftime('%Y-%m-%d 23:59:59'),'%Y-%m-%d %H:%M:%S')

In [16]:
# 获取当前topic有多少个分区 
cluster_data = c.list_topics(topic = kafka['topicData'])
topic_data = cluster_data.topics[kafka['topicData']]
available_partitions = topic_data.partitions

In [17]:
start_tps = [TopicPartition(kafka['topicData'], index, int(start_time.timestamp() * 1000)) for index in range(len(available_partitions))]
start_offset = c.offsets_for_times(start_tps)
end_tps = [TopicPartition(kafka['topicData'], index, int(end_time.timestamp() * 1000)) for index in range(len(available_partitions))]
end_offset = c.offsets_for_times(end_tps)
c.assign(start_offset)

In [18]:
start_offset

[TopicPartition{topic=dc_abnormal_data,partition=0,offset=-1,error=None},
 TopicPartition{topic=dc_abnormal_data,partition=1,offset=38239,error=None},
 TopicPartition{topic=dc_abnormal_data,partition=2,offset=-1,error=None}]

In [19]:
values = []
while True:
    msg = c.poll(100) # poll：轮询一次过程中，在一定时间内broker可消费的数据，单位为ms，如这里的100,就是在0.1s内拉取数据返回到消费者端
    if msg is None:
        continue
    if msg.error():
        prinit('Consumer error:{}'.format(msg.error()))
        continue
    offset = msg.offset()
    if offset < end_offset[1].offset: # 有数据的在partition为1的topicpartition
        kafka_timestamp = msg.timestamp()[1] # 搞数据入kafka的时间戳
        kafka_value = json.loads(msg.value())
        values.append(kafka_value)
    else: 
        c.unassign()  # 超过当前Partition的话，停止订阅
        break
c.close()

In [22]:
end_offset[1].offset

38899

In [9]:
values_to_json = {}
values_to_json['values'] = values

In [10]:
with open('../values.json', 'w') as f:
    json.dump(values_to_json, f, indent = 2, sort_keys = True, ensure_ascii = False)

________

### 从多个partition中取数

In [1]:
from confluent_kafka import Consumer, KafkaError, TopicPartition, Message
import datetime, time, json

In [2]:
kafka = {}
kafka['http'] = '192.168.50.107:19092'
kafka['topicCtrl'] = 'dc_abnormal_control'
kafka['topicData'] = 'dc_test_prophet'
kafka['offset'] = 'earliest'
kafka['groupid'] = 'AnomalyDetectTest01'

In [3]:
c = Consumer({
    'bootstrap.servers':kafka['http'], # kafka所在ip地址
    'group.id':time.time(),
    'enable.auto.commit':True, # 是否自动提交offset，设为True时，每隔一段时间就会提交一次offset
    'default.topic.config':{
        'auto.offset.reset':kafka['offset']
    }  
})

In [4]:
# 拉取昨天一天的数据，start_time、end_time这两个时间可以随便设置
start = datetime.datetime.now() - datetime.timedelta(days=40)
start_time = datetime.datetime.strptime(start.strftime('%Y-%m-%d 00:00:00'),'%Y-%m-%d %H:%M:%S')
end = datetime.datetime.now() - datetime.timedelta(days=40)
end_time = datetime.datetime.strptime(end.strftime('%Y-%m-%d 23:59:59'),'%Y-%m-%d %H:%M:%S')

In [5]:
# 获取当前topic有多少个分区 
cluster_data = c.list_topics(topic = kafka['topicData'])
topic_data = cluster_data.topics[kafka['topicData']]
available_partitions = topic_data.partitions

In [6]:
start_tps = [TopicPartition(kafka['topicData'], index, int(start_time.timestamp() * 1000)) for index in range(len(available_partitions))]
start_offset = c.offsets_for_times(start_tps)
end_tps = [TopicPartition(kafka['topicData'], index, int(end_time.timestamp() * 1000)) for index in range(len(available_partitions))]
end_offset = c.offsets_for_times(end_tps)
c.assign(start_offset)

In [7]:
end_offset

[TopicPartition{topic=dc_test_prophet,partition=0,offset=156,error=None},
 TopicPartition{topic=dc_test_prophet,partition=1,offset=130,error=None},
 TopicPartition{topic=dc_test_prophet,partition=2,offset=168,error=None}]

In [9]:
values = []
while True:
    msg = c.poll(100) # poll：轮询一次过程中，在一定时间内broker可消费的数据，单位为ms，如这里的100,就是在0.1s内拉取数据返回到消费者端
    if msg is None:
        continue
    if msg.error():
        prinit('Consumer error:{}'.format(msg.error()))
        continue
    offset = msg.offset()
    if offset < end_offset[0].offset: 
        kafka_timestamp = msg.timestamp()[1]/1000 # 搞数据入kafka的时间戳
        kafka_value = json.loads(msg.value())
        values.append(kafka_value)
    else: 
        c.unassign()  # 超过当前Partition的话，停止订阅
        break
c.close()

In [11]:
len(values)

0

In [ ]:
values = []
while True:
    msg = c.poll(100) # poll：轮询一次过程中，在一定时间内broker可消费的数据，单位为ms，如这里的100,就是在0.1s内拉取数据返回到消费者端
    if msg is None:
        continue
    if msg.error():
        prinit('Consumer error:{}'.format(msg.error()))
        continue
    offset = msg.offset()
    datapartitions = [i for i in end_offset if i.offset != -1]

    for i in datapartitions:
        if offset < i.offset: 
            kafka_timestamp = msg.timestamp()[1] # 搞数据入kafka的时间戳
            kafka_value = json.loads(msg.value())
            values.append(kafka_value)
        else: 
            c.unassign()  # 超过当前Partition的话，停止订阅
            break
c.close()

In [12]:
msg = c.poll(100)
msg.timestamp()

(1, 1649401570230)

In [10]:
len(values)

0

In [11]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(normalize=True)

# 创建数据集
import numpy as np
x = np.arange(10)    
y = 2 * x + 1    
X = x[:, np.newaxis]    
model.fit( X, y )

# 输出结果
print( model.coef_ )    
print( model.intercept_ )

[2.]
0.9999999999999982


/home/weijunfei/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  FutureWarning,


In [7]:
OHE = OneHotEncoder()    
OHE.fit_transform( enc_DF ).toarray()

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])